In [1]:
import os
import pandas as pd
from datetime import datetime 

In [2]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

current_dir:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


In [3]:
from se4g_helper import build_dataframe, update_dataset

In [4]:
df_columns=['station_code', 'station_name', 'station_altitude', 'network_countrycode', 'pollutant', 'value_datetime_begin',
            'value_datetime_end', 'value_datetime_updated', 'value_numeric', 'samplingpoint_x', 'samplingpoint_y'
            ]
data_type = ['VARCHAR', 'VARCHAR', 'FLOAT', 'CHAR(2)', 'VARCHAR', 'VARCHAR',
             'VARCHAR', 'VARCHAR', 'FLOAT', 'FLOAT', 'FLOAT'
             ]

Create se4g_pollution_main.csv

In [5]:
fileName = "se4g_pollution_dataset.csv" # execution time: 1m23s -> 1m51s
folder_out = 'data_prova'


columns = df_columns

data_type = data_type


for item_name in os.listdir(folder_out):
    item_path = os.path.join(folder_out, item_name)

    if os.path.isdir(item_path):
        new_df = build_dataframe(item_name, folder_out = folder_out )

        update_dataset(new_df, folder_out = folder_out, fileName = "se4g_pollution_main.csv")

AD_SO2.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AD_SO2.csv' mode='r' encoding='utf-8-sig'>
AD_NO.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AD_NO.csv' mode='r' encoding='utf-8-sig'>
AD_NO2.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AD_NO2.csv' mode='r' encoding='utf-8-sig'>
AD_CO.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AD_CO.csv' mode='r' encoding='utf-8-sig'>
AD_PM10.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AD_PM10.csv' mode='r' encoding='utf-8-sig'>
AT_SO2.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AT_SO2.csv' mode='r' encoding='utf-8-sig'>
AT_NO.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AT_NO.csv' mode='r' encoding='utf-8-sig'>
AT_NO2.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AT_NO2.csv' mode='r' encoding='utf-8-sig'>
AT_CO.csv
<_io.TextIOWrapper name='data_prova\\01-06-2023_10_39_44\\AT_CO.csv' mode='r' encoding='utf-8-sig'

Create se4g_pollution_dashboard.csv

In [3]:
fileName = "se4g_pollution_main.csv" # execution time: 1m23s -> 1m51s
folder_out = 'data_prova'
full_path = os.path.join(folder_out, fileName)

columns = ['pollutant', 'country', 'month_day', 'value_numeric_mean', 'value_datetime_begin']

country = {'AD': 'Andorra', 'SE': 'Sweden', 'DE': 'Germany', 'CY': 'Undefined', 'BE': 'Belgium',
        'FI': 'Finland', 'ES': 'Spain', 'CZ': 'Czech Republic', 'BG': 'Bulgaria', 'BA': 'Bosnia and Herzegovina',
        'EE': 'Estonia', 'CH': 'Switzerland', 'AT': 'Austria', 'DK': 'Denmark'}
   
if os.path.isfile(full_path):
    dashboard_df = pd.read_csv(full_path)
    # Convert 'value_datetime_end' to datetime objects and extract the day
    datetime_objects = dashboard_df['value_datetime_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))
    
    dashboard_df['month_day'] = datetime_objects.apply(lambda x: x.strftime('%m%d'))
    
    dashboard_df['value_datetime_begin'] = pd.to_datetime(dashboard_df['value_datetime_begin']).dt.strftime('%Y-%m-%d %H:%M:%S')

    # Compute daily mean of 'value_numeric' for each 'pollutant' and 'network_countrycode'
    daily_mean = dashboard_df.groupby(['pollutant', 'network_countrycode', 'month_day'])['value_numeric'].mean().reset_index()
    
    # Merge the daily mean back to the original dataframe
    dashboard_df = dashboard_df.merge(daily_mean, on=['pollutant', 'network_countrycode', 'month_day'], suffixes=('', '_mean'))    

    dashboard_df['country'] = dashboard_df['network_countrycode'].map(country)
    dashboard_df = dashboard_df[columns].copy()

    dashboard_df = dashboard_df.drop_duplicates().reset_index(drop=True)
    dashboard_df = dashboard_df.sort_values('month_day')
    dashboard_df.to_csv('data_prova/se4g_dashboard_dataset.csv', index=False)
    print("Dataset ",full_path," created")
    
else: 
        print("Dataset ",full_path," does not exist")
        print(os.getcwd())

Dataset  data_prova\se4g_pollution_dataset.csv  created


In [12]:
file_path = 'data_prova/31-05-2023_09_15_34/ES_PM10.csv'
file_path = 'data_prova/31-05-2023_10_53_20/ES_PM10.csv'
print(file_path)

with open(file_path, 'r') as file:
    print(file)
    first_line = file.readline().strip()

if not first_line.startswith('<!DOCTYPE html'): 
    #print(fileName,'exist')

    df_temp = pd.read_csv(file_path)
    print(df_temp['network_countrycode'].head())

data_prova/31-05-2023_10_53_20/ES_PM10.csv
<_io.TextIOWrapper name='data_prova/31-05-2023_10_53_20/ES_PM10.csv' mode='r' encoding='utf-8'>
0    ES
1    ES
2    ES
3    ES
4    ES
Name: network_countrycode, dtype: object


In [ ]:
import requests
def download_request(COUNTRIES= 'ES',
                        POLLUTANTS= 'PM10',
                        folder_out = 'data_prova'):
    print ('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"


    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir,'directory created')
        
    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            #Download and save to local path
            print('Downloading: %s' % downloadFile )

            file = requests.get(downloadFile).content
            full_file = os.path.join(folder_out, dir, fileName)
			


            '''with codecs.open(full_file, 'r', encoding='utf-8-sig') as file:
                content = file.read()'''

            # Modify the content as needed
            with codecs.open(full_file, 'w', encoding='utf-8') as output:
                output.write(file)



            '''output = open(full_file, 'wb')
            output.write(file)
            output.close()'''
            print ('Saved locally as: %s ' % full_file)
            print ('-----')
    print ('Download finished')
    return dir

dirr = download_request()

In [14]:
def download_request(COUNTRIES=['ES'], POLLUTANTS=['PM10'], folder_out='data_prova'):
    print('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"

    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir, 'directory created')

    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            # Download and save to local path
            print('Downloading: %s' % downloadFile)

            file_content = requests.get(downloadFile).content
            file_content_str = file_content.decode('utf-8-sig')

            full_file = os.path.join(folder_out, dir, fileName)

            with open(full_file, 'w', encoding='utf-8') as output:
                output.write(file_content_str)

            print('Saved locally as: %s ' % full_file)
            print('-----')
    print('Download finished')
    return dir

dirr = download_request()

-----------------------------------------------------------------------
31-05-2023_11_47_40 directory created
Downloading: http://discomap.eea.europa.eu/map/fme/latest/ES_PM10.csv
Saved locally as: data_prova\31-05-2023_11_47_40\ES_PM10.csv 
-----
Download finished


In [15]:
def download_request(COUNTRIES=['ES'], POLLUTANTS=['PM10'], folder_out='data_prova'):
    print('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"

    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir, 'directory created')

    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            # Download and save to local path
            print('Downloading: %s' % downloadFile)

            file_content = requests.get(downloadFile).content
            file_content_str = file_content.decode('utf-8-sig')

            full_file = os.path.join(folder_out, dir, fileName)

            with codecs.open(full_file, 'w', encoding='utf-8') as output:
                output.write(file_content_str)

            print('Saved locally as: %s ' % full_file)
            print('-----')
    print('Download finished')
    return dir

dirr = download_request()

-----------------------------------------------------------------------
31-05-2023_11_47_59 directory created
Downloading: http://discomap.eea.europa.eu/map/fme/latest/ES_PM10.csv
Saved locally as: data_prova\31-05-2023_11_47_59\ES_PM10.csv 
-----
Download finished
